Which k-mers are contributing to the matches? Realistically, I want to be able to show this for every protein match

# Imports and setup

### Auto-re-import python modules, useful for editing local files

In [2]:
%load_ext autoreload
%autoreload 2

## Imports

In [79]:
import os

import polars as pl
import rich
import screed

import sourmash

# Read in data

In [4]:
ksize = 27
moltype = "hp"

In [5]:
cleaned_multisearch_folder = "/home/ec2-user/data/seanome-kmerseek/scope-benchmark/analysis-outputs/hp/00_cleaned_multisearch_results"

pq = os.path.join(
    cleaned_multisearch_folder, f"scope40.multisearch.{moltype}.k{ksize}.filtered.pq"
)
multisearch = pl.read_parquet(pq)
multisearch

query_name,query_md5,match_name,match_md5,containment,max_containment,jaccard,intersect_hashes,prob_overlap,prob_overlap_adjusted,containment_adjusted,containment_adjusted_log10,tf_idf_score,query_scop_id,query_scop_lineage,query_scop_lineage_fixed,query_family,query_superfamily,query_fold,query_class,match_scop_id,match_scop_lineage,match_scop_lineage_fixed,match_family,match_superfamily,match_fold,match_class,same_family,same_superfamily,same_fold,same_class,__index_level_0__
str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,cat,cat,cat,cat,str,str,str,cat,cat,cat,cat,bool,bool,bool,bool,i64
"""d1g0da3 b.1.5.1 (A:584-684) Tr…","""5f9e6eb4503a7471cfc700032d52e9…","""d3gcea_ b.33.1.0 (A:) automate…","""e0f4b134db68bb57a20d17b9ac232e…",0.04,0.04,0.02,3.0,2.0653e-12,0.000476,84.083414,1.92471,0.38116,"""d1g0da3""","""b.1.5.1""","""b.1.5.1""","""b.1.5.1""","""b.1.5""","""b.1""","""b""","""d3gcea_""","""b.33.1.0""","""b.33.1.0""","""b.33.1.0""","""b.33.1""","""b.33""","""b""",false,false,false,true,2
"""d5mdmc_ e.76.1.0 (C:) automate…","""e5ee23ea824b28a30b086d141d3b1f…","""d1gd8a_ d.188.1.1 (A:) Prokary…","""fc8ac3e7f13b9f98a58e035e210d24…",0.007916,0.037975,0.006593,3.0,2.0653e-12,0.000476,16.639198,1.221132,0.075427,"""d5mdmc_""","""e.76.1.0""","""e.76.1.0""","""e.76.1.0""","""e.76.1""","""e.76""","""e""","""d1gd8a_""","""d.188.1.1""","""d.188.1.1""","""d.188.1.1""","""d.188.1""","""d.188""","""d""",false,false,false,false,5
"""d4f3mb_ d.58.53.2 (B:) CRISPR-…","""09df7885164ac177215e7b233602b1…","""d1dlwa_ a.1.1.1 (A:) Protozoan…","""1f16d4c0915a423839520e55696031…",0.019231,0.044444,0.013605,4.0,2.7537e-12,0.000634,30.318539,1.481708,0.18325,"""d4f3mb_""","""d.58.53.2""","""d.58.53.2""","""d.58.53.2""","""d.58.53""","""d.58""","""d""","""d1dlwa_""","""a.1.1.1""","""a.1.1.1""","""a.1.1.1""","""a.1.1""","""a.1""","""a""",false,false,false,false,7
"""d3j9ca1 b.179.1.1 (A:174-225) …","""f78b59c2f05c2ea1abde77ccf2e0ed…","""d3tewa2 b.179.1.1 (A:15-225) P…","""24efe83caff5ff87a2cfcf29349e23…",1.0,1.0,0.140541,26.0,1.7899e-11,0.004123,242.548311,2.384798,9.52899,"""d3j9ca1""","""b.179.1.1""","""b.179.1.1""","""b.179.1.1""","""b.179.1""","""b.179""","""b""","""d3tewa2""","""b.179.1.1""","""b.179.1.1""","""b.179.1.1""","""b.179.1""","""b.179""","""b""",true,true,true,true,9
"""d3c3ka1 c.93.1.0 (A:7-277) aut…","""6b8ab65427e44ceb9584ee9f5377c4…","""d3tewa2 b.179.1.1 (A:15-225) P…","""24efe83caff5ff87a2cfcf29349e23…",0.008163,0.010811,0.004673,2.0,1.3768e-12,0.000317,25.739821,1.410606,0.077788,"""d3c3ka1""","""c.93.1.0""","""c.93.1.0""","""c.93.1.0""","""c.93.1""","""c.93""","""c""","""d3tewa2""","""b.179.1.1""","""b.179.1.1""","""b.179.1.1""","""b.179.1""","""b.179""","""b""",false,false,false,false,10
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""d1sqja1 b.69.13.1 (A:4-430) Ol…","""07c9fb70f433c78f0342775b5a899a…","""d2j9la1 d.37.1.1 (A:578-746) C…","""8e424789ddb92cb84f023eb4f77b13…",0.007481,0.020979,0.005545,3.0,2.0653e-12,0.000476,15.726324,1.196627,0.071289,"""d1sqja1""","""b.69.13.1""","""b.69.13.1""","""b.69.13.1""","""b.69.13""","""b.69""","""b""","""d2j9la1""","""d.37.1.1""","""d.37.1.1""","""d.37.1.1""","""d.37.1""","""d.37""","""d""",false,false,false,false,43818
"""d1dusa_ c.66.1.4 (A:) Hypothet…","""c59d01b63986fe2e6ae3cc70e4310a…","""d2j9la1 d.37.1.1 (A:578-746) C…","""8e424789ddb92cb84f023eb4f77b13…",0.017857,0.020979,0.00974,3.0,2.0653e-12,0.000476,37.537239,1.574462,0.170161,"""d1dusa_""","""c.66.1.4""","""c.66.1.4""","""c.66.1.4""","""c.66.1""","""c.66""","""c""","""d2j9la1""","""d.37.1.1""","""d.37.1.1""","""d.37.1.1""","""d.37.1""","""d.37""","""d""",false,false,false,false,43819
"""d1ynfa1 d.126.1.7 (A:2-441) Su…","""62ace1b97b6d76d8eabadd41c586f5…","""d2j9la1 d.37.1.1 (A:578-746) C…","""8e424789ddb92cb84f023eb4f77b13…",0.004831,0.013986,0.003604,2.0,1.3768e-12,0.000317,15.232503,1.182771,0.046034,"""d1ynfa1""","""d.126.1.7""","""d.126.1.7""","""d.126.1.7""","""d.126.1""","""d.126

In [114]:
sig_folder = "/home/ec2-user/data/seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__hp_k20-60/sourmash/sigs"

sigfile = os.path.join(
    sig_folder,
    f"query.astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.{moltype}.k{ksize}.sig.zip",
)

sigs = {x.name.split()[0]: x for x in sourmash.load_file_as_signatures(sigfile)}
len(sigs)

15177

In [115]:
query_scop_id_index = multisearch.columns.index("query_scop_id")
match_scop_id_index = multisearch.columns.index("match_scop_id")

for row in multisearch.iter_rows():
    query_id = row[query_scop_id_index]
    match_id = row[match_scop_id_index]
    query_sig = sigs[query_id]
    match_sig = sigs[match_id]
    # overlap = query_sig.
    break

In [116]:
# query_sig.minhash.intersection(match_sig.minhash)

In [117]:
def overlap(sig1, sig2):
    """
    provide detailed comparison of two signatures

    Cribbed from: https://github.com/sourmash-bio/sourmash/blob/2cc44e026bc147a292a1dcac3ab2bcd046be09fa/src/sourmash/sig/__main__.py#L379C1-L457C1
    """

    try:
        similarity = sig1.similarity(sig2)
    except ValueError:
        raise

    cont1 = sig1.contained_by(sig2)
    cont2 = sig2.contained_by(sig1)

    name1 = sig1.name
    name2 = sig2.name

    md5_1 = sig1.md5sum()
    md5_2 = sig2.md5sum()

    ksize = sig1.minhash.ksize
    moltype = sig1.minhash.moltype

    num = sig1.minhash.num
    size1 = len(sig1.minhash)
    size2 = len(sig2.minhash)

    scaled = sig1.minhash.scaled

    hashes_1 = set(sig1.minhash.hashes)
    hashes_2 = set(sig2.minhash.hashes)

    num_common = len(hashes_1 & hashes_2)
    disjoint_1 = len(hashes_1 - hashes_2)
    disjoint_2 = len(hashes_2 - hashes_1)
    num_union = len(hashes_1.union(hashes_2))

    print(
        """\
first signature:
  signature: {name1}
  md5: {md5_1}
  k={ksize} molecule={moltype} num={num} scaled={scaled}

second signature:
  signature: {name2}
  md5: {md5_2}
  k={ksize} molecule={moltype} num={num} scaled={scaled}

similarity:                  {similarity:.5f}
first contained in second:   {cont1:.5f}
second contained in first:   {cont2:.5f}

number of hashes in first:   {size1}
number of hashes in second:  {size2}

number of hashes in common:  {num_common}
only in first:               {disjoint_1}
only in second:              {disjoint_2}
total (union):               {num_union}
""".format(
            **locals()
        )
    )


overlap(query_sig, match_sig)

first signature:
  signature: d1g0da3 b.1.5.1 (A:584-684) Transglutaminase, two C-terminal domains {Red sea bream (Chrysophrys major) [TaxId: 143350]}
  md5: 5f9e6eb4503a7471cfc700032d52e90e
  k=27 molecule=hp num=0 scaled=1

second signature:
  signature: d3gcea_ b.33.1.0 (A:) automated matches {Nocardioides aromaticivorans [TaxId: 200618]}
  md5: e0f4b134db68bb57a20d17b9ac232e91
  k=27 molecule=hp num=0 scaled=1

similarity:                  0.02498
first contained in second:   0.04000
second contained in first:   0.03846

number of hashes in first:   75
number of hashes in second:  78

number of hashes in common:  3
only in first:               72
only in second:              75
total (union):               150



### Read fasta sequences

In [118]:
fasta = "/home/ec2-user/data/seanome-kmerseek/scope-benchmark/rawdata/astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.fa"

sequences = {}

with screed.open(fasta) as records:
    for record in records:
        name = record["name"].split()[0]
        sequences[name] = record["sequence"].upper()
len(sequences)

15177

In [119]:
# ! head $fasta

In [120]:
match_id

'd3gcea_'

In [121]:
query_sig.minhash.moltype

'hp'

In [122]:
query_seq = sequences[query_id]
match_seq = sequences[match_id]
# list(query_sig.minhash.kmers_and_hashes(match_seq, is_protein=True))

In [123]:
from sig2kmer import degenerate_protein_chatgpt

In [124]:
def get_is_protein(sig):
    is_protein = False
    if sig.minhash.moltype != "DNA":
        is_protein = True
    return is_protein


def add_sequence(minhash, sequence, is_protein):
    if is_protein:
        minhash.add_protein(sequence)
    else:
        minhash.add_sequence(sequence)


def sequence_kmers_in_sig(sig, sequence):
    """Return k-mers from sequence present in signature k-mers"""
    # Make a minhash just for looking through the sequence
    query_mh = sig.minhash.flatten()
    seq_mh = query_mh.copy_and_clear()

    # figure out protein vs dna
    is_protein = get_is_protein(sig)
    add_sequence(seq_mh, sequence, is_protein)

    intersecting = query_mh.intersection(seq_mh)
    if intersecting:
        # Found matching k-mers!

        kmers_hashes = intersecting.kmers_and_hashes(
            sequence, force=False, is_protein=is_protein
        )
        found_kmers_hashes = [
            (i, degenerate_protein_chatgpt(kmer, sig.minhash.moltype), kmer, hashval)
            for i, (kmer, hashval) in enumerate(kmers_hashes)
            if hashval in intersecting.hashes
        ]
        return found_kmers_hashes

    else:
        return []


sequence_kmers_in_sig(query_sig, match_seq)

[(74,
  'phhhphhhphhhhphhphhhphpphhh',
  'TGAPTALPCVLPVRAYDVVVDGTEILV',
  1377032650408549891),
 (75,
  'hhhphhhphhhhphhphhhphpphhhh',
  'GAPTALPCVLPVRAYDVVVDGTEILVA',
  9687257153334895900),
 (76,
  'hhphhhphhhhphhphhhphpphhhhh',
  'APTALPCVLPVRAYDVVVDGTEILVAP',
  4002281547216460058)]

In [125]:
sequence_kmers_in_sig(match_sig, query_seq)

[(2,
  'phhhphhhphhhhphhphhhphpphhh',
  'ELLVQVPGKAVVWEPLTAYVSFTNPLP',
  1377032650408549891),
 (3,
  'hhhphhhphhhhphhphhhphpphhhh',
  'LLVQVPGKAVVWEPLTAYVSFTNPLPV',
  9687257153334895900),
 (4,
  'hhphhhphhhhphhphhhphpphhhhh',
  'LVQVPGKAVVWEPLTAYVSFTNPLPVP',
  4002281547216460058)]

In [126]:
query_seq

'TPELLVQVPGKAVVWEPLTAYVSFTNPLPVPLKGGVFTLEGAGLLSATQIHVNGAVAPSGKVSVKLSFSPMRTGVRKLLVDFDSDRLKDVKGVTTVVVHKK'

In [127]:
degenerate_protein_chatgpt(query_seq, "hp")

'phphhhphhhphhhhphhphhhphpphhhhhhphhhhphphhhhhphpphphphhhhhphphphphphphhpphhpphhhphpppphpphphhpphhhppp'

In [128]:
match_seq

'STPVRVATLDQLKPGVPTAFDVDGDEVMVVRDGDSVYAISNLCSHAEAYLDMGVFHAESLEIECPLHVGRFDVRTGAPTALPCVLPVRAYDVVVDGTEILVAPK'

In [129]:
degenerate_protein_chatgpt(match_seq, "hp")

'pphhphhphpphphhhhphhphphpphhhhpphpphhhhpphppphphhhphhhhphpphphpphhphhphphpphhhphhhphhhhphhphhhphpphhhhhp'

In [130]:
len(query_seq)

101

In [131]:
query_sig.name

'd1g0da3 b.1.5.1 (A:584-684) Transglutaminase, two C-terminal domains {Red sea bream (Chrysophrys major) [TaxId: 143350]}'

In [132]:
match_sig.name

'd3gcea_ b.33.1.0 (A:) automated matches {Nocardioides aromaticivorans [TaxId: 200618]}'

## Write class and code to show overlap between two sequences

In [185]:
from dataclasses import dataclass
from typing import List, Optional, Tuple

from rich.console import Console
from rich.theme import Theme

custom_theme = Theme({"overlap": "bold white on blue"})


def get_to_add(encoded, protein, use_encoded):
    if use_encoded:
        to_add = encoded
    else:
        to_add = protein
    return to_add


def add_to_stitched(stitched, prev_i, to_add):
    if prev_i == None:
        stitched = to_add
    else:
        stitched += to_add[-1]
    return stitched


def stitch_kmers(overlap, use_encoded=False):
    """
    use_encoded: whether or not to use the encoded protein sequences, e.g. the HP version and not the original protein
    """

    prev_i = None

    stitched = ""
    for i, encoded, protein, hashval in overlap:
        if prev_i == None or i == prev_i + 1:
            # if first or sequential
            to_add = get_to_add(encoded, protein, use_encoded)
            stitched = add_to_stitched(stitched, prev_i, to_add)
            prev_i = i
        else:
            raise ValueError(
                f"Non-sequential indices -- Previous index: {prev_i}, this index: {i}"
            )
    return stitched


class KmerStitcher:
    @staticmethod
    def get_to_add(encoded: str, protein: str, use_encoded: bool) -> str:
        return encoded if use_encoded else protein

    @staticmethod
    def add_to_stitched(stitched: str, prev_i: Optional[int], to_add: str) -> str:
        return to_add if prev_i is None else stitched + to_add[-1]

    @classmethod
    def stitch_kmers(cls, overlap: List[Tuple], use_encoded: bool = False) -> str:
        """Stitches kmers together based on overlap information"""
        prev_i = None
        stitched = ""

        for i, encoded, protein, hashval in overlap:
            if prev_i is None or i == prev_i + 1:
                to_add = cls.get_to_add(encoded, protein, use_encoded)
                stitched = cls.add_to_stitched(stitched, prev_i, to_add)
                prev_i = i
            else:
                raise ValueError(
                    f"Non-sequential indices -- Previous: {prev_i}, current: {i}"
                )

        return stitched


@dataclass
class OverlapInfo:
    """Stores information about sequence overlap"""

    index: int
    overlap_seq: str
    overlap_encoded: str
    overlap_length: int


class SigSeq:
    """Sourmash Signature with Sequence"""

    def __init__(self, sig: sourmash.SourmashSignature, seq: str):
        self.sig = sig
        self.seq = seq

        # Returns None if no encoding
        self.seq_encoded = degenerate_protein_chatgpt(seq, sig.minhash.moltype)

    def get_overlapping_kmers(self, other: SigSeq) -> tuple:
        return sequence_kmers_in_sig(self.sig, other.seq)

    @staticmethod
    def _get_display_padding(self_overlap, other_overlap):
        if self_overlap[0][0] > other_overlap[0][0]:
            self_left_pad = (self_overlap[0][0] - other_overlap[0][0]) * " "
            other_left_pad = ""
            add_to_match_string = " " * other_overlap[0][0]
        else:
            self_left_pad = ""
            other_left_pad = (other_overlap[0][0] - self_overlap[0][0]) * " "
            add_to_match_string = " " * self_overlap[0][0]
        return self_left_pad, other_left_pad, add_to_match_string

    @staticmethod
    def make_rich_overlap_string(left_pad, seq, i_overlap, overlap_length):
        seq_left = seq[:i_overlap]
        overlap_seq = seq[
            i_self_until_overlap : (i_self_until_overlap + overlap_length)
        ]
        seq_right = seq[(i_overlap + overlap_length) :]
        return f"{left_pad}{seq_left}[overlap]{overlap_seq}[/overlap]{seq_right}"

    def _verify_overlap(self) -> None:
        """Verifies that overlap sequences match within the main sequence"""
        start = self.overlap_info.index
        end = start + self.overlap_info.overlap_length
        assert self.overlap_info.overlap_seq == self.seq[start:end]
        assert self.overlap_info.overlap_encoded == self.seq_encoded[start:end]

    def compute_overlap(self, other: SigSeq) -> None:
        """Computes overlap information between two sequences"""
        overlap = self.get_overlapping_kmers(other)
        overlap_seq = KmerStitcher.stitch_kmers(overlap, use_encoded=False)
        overlap_encoded = KmerStitcher.stitch_kmers(overlap, use_encoded=True)

        index = self.seq.index(overlap_seq)
        overlap_length = overlap[-1][0] - overlap[0][0] + self.sig.minhash.ksize

        self.overlap_info = OverlapInfo(
            index, overlap_seq, overlap_encoded, overlap_length
        )

    @staticmethod
    def calculate_padding(
        self_overlap: OverlapInfo, other_overlap: OverlapInfo
    ) -> Tuple[str, str, str]:
        """Calculates display padding for alignment visualization"""
        if self_overlap.index > other_overlap.index:
            self_pad = " " * (self_overlap.index - other_overlap.index)
            other_pad = ""
            match_pad = " " * other_overlap.index
        else:
            self_pad = ""
            other_pad = " " * (other_overlap.index - self_overlap.index)
            match_pad = " " * self_overlap.index

        return self_pad, other_pad, match_pad

    def display_alignment(self, other):
        """Displays the alignment between two sequences"""
        # Compute overlaps
        self.compute_overlap(other)
        other.compute_overlap(self)

        # Verify overlaps
        self._verify_overlap()
        other._verify_overlap()

        # Calculate padding
        self_pad, other_pad, match_pad = self.calculate_padding(
            self.overlap_info, other.overlap_info
        )
        # self.overlap = self.get_overlapping_kmers(other)
        # other.overlap = other.get_overlapping_kmers(self)

        # # Use the "self" to find which kmers in the "other" overlap
        # other.overlap_seq = stitch_kmers(self.overlap, use_encoded=False)
        # other.overlap_encoded = stitch_kmers(self.overlap, use_encoded=True)

        # # Use the "other" seq to find which kmers in the "self" overlap
        # self.overlap_seq = stitch_kmers(other.overlap, use_encoded=False)
        # self.overlap_encoded = stitch_kmers(other.overlap, use_encoded=True)

        # other.i_overlap = other.seq.index(other_overlap_seq)
        # self.i_overlap = self.seq.index(self_overlap_seq)

        # overlap_length = (
        #     self_overlap[-1][0] - self_overlap[0][0] + query.sig.minhash.ksize
        # )
        # self.overlap_length = overlap_length
        # other.overlap_length = overlap_length
        # print("\n---\noverlap_length", overlap_length)

        # self.check_all_overlap_seqs(self, other, overlap_length)

        # self_left_pad, other_left_pad, add_to_match_string = self._get_display_padding(
        #     self_overlap, other_overlap
        # )

        # Maybe something other than "match"? Seems overloaded of a word here
        # match_string = (
        #     self_left_pad + other_left_pad + add_to_match_string + "|" * overlap_length
        # )
        match_string = match_pad + "|" * self.overlap_info.overlap_length

        console = Console(theme=custom_theme)
        print(self.sig.name)
        # print(f"{self_left_pad}{self.seq}")
        # console.print(
        #     f"{self_left_pad}{self.seq[:i_self_until_overlap]}[overlap]{self_overlap_seq}[/overlap]{self.seq[(i_self_until_overlap + overlap_length):]}",
        #     end="",
        # )
        console.print(self._get_display_sequence(self_pad))

        console.print(f"{self_left_pad}{self.seq_encoded}")
        print(match_string)
        print(f"{other_left_pad}{other.seq_encoded}")
        print(f"{other_left_pad}{other.seq}")
        print(other.sig.name)

Writing sigseq.py


In [204]:
from sigseq import SigSeq

query = SigSeq(query_sig, query_seq)
match = SigSeq(match_sig, match_seq)

query.display_alignment(match)

d1g0da3 b.1.5.1 (A:584-684) Transglutaminase, two C-terminal domains {Red sea bream (Chrysophrys major) [TaxId: 
143350]}

TPELLVQVPGKAVVWEPLTAYVSFTNPLPVPLKGGVFTLEGAG
LLSATQIHVNGAVAPSGKVSVKLSFSPMRTGVRKLLVDFDSDRLKDVKGVTTVVVHKK
                                                                        phphhhphhhphhhhphhphhhphpphhhhhhphhhhphphhh
hhphpphphphhhhhphphphphphphhpphhpphhhphpppphpphphhpphhhppp

|||||||||||||||||||||||||||||

pphhphhphpphphhhhphhphphpphhhhpphpphhhhpphppphphhhphhhhphpphphpphhphhphphpphhhphhhphhhhphhphhhphpphhhhhp

STPVRVATLDQLKPGVPTAFDVDGDEVMVVRDGDSVYAISNLCSHAEAYLDMGVFHAESLEIECPLHVGRFDVRTGAPTALPCVLPVRAYDVVVDGTEILVAPK

d3gcea_ b.33.1.0 (A:) automated matches {Nocardioides aromaticivorans [TaxId: 200618]}

In [184]:
%debug

> /tmp/ipykernel_6267/735794354.py(132)compute_overlap()
    130         overlap_encoded = KmerStitcher.stitch_kmers(overlap, use_encoded=True)
    131 
--> 132         index = self.seq.index(overlap_seq)
    133         overlap_length = overlap[-1][0] - overlap[0][0] + self.sig.minhash.ksize
    134 



ipdb>  dir()


['other', 'overlap', 'overlap_encoded', 'overlap_seq', 'self']


ipdb>  overlap_seq


'TGAPTALPCVLPVRAYDVVVDGTEILVAP'


ipdb>  self.seq


'TPELLVQVPGKAVVWEPLTAYVSFTNPLPVPLKGGVFTLEGAGLLSATQIHVNGAVAPSGKVSVKLSFSPMRTGVRKLLVDFDSDRLKDVKGVTTVVVHKK'


ipdb>  q


In [156]:
console = Console()

In [158]:
console.print?

Signature:
console.print(
    *objects: Any,
    sep: str = ' ',
    end: str = '\n',
    style: Union[rich.style.Style, str, NoneType] = None,
    justify: Optional[Literal['default', 'left', 'center', 'right', 'full']] = None,
    overflow: Optional[Literal['fold', 'crop', 'ellipsis', 'ignore']] = None,
    no_wrap: Optional[bool] = None,
    emoji: Optional[bool] = None,
    markup: Optional[bool] = None,
    highlight: Optional[bool] = None,
    width: Optional[int] = None,
    height: Optional[int] = None,
    crop: bool = True,
    soft_wrap: Optional[bool] = None,
    new_line_start: bool = False,
) -> None
Docstring:
Print to the console.

Args:
    objects (positional args): Objects to log to the terminal.
    sep (str, optional): String to write between print data. Defaults to " ".
    end (str, optional): String to write at end of print data. Defaults to "\\n".
    style (Union[str, Style], optional): A style to apply to output. Defaults to None.
    justify (str, optional):

In [148]:
%debug

> /tmp/ipykernel_6267/1524215248.py(82)display_alignment()
     80 
     81         other_until_overlap = other.seq.index(other_overlap_seq)
---> 82         self_until_overlap = self.seq.index(self_overlap_seq)
     83 
     84         overlap_length = (



ipdb>  self_overlap_seq


'TGAPTALPCVLPVRAYDVVVDGTEILVAP'


ipdb>  other_overlap_seq


'TGAPTALPCVLPVRAYDVVVDGTEILVAP'


ipdb>  q


In [134]:
query.sig.minhash.ksize

27

In [135]:
self_overlap = query.get_overlapping_kmers(match)
self_overlap

[(74,
  'phhhphhhphhhhphhphhhphpphhh',
  'TGAPTALPCVLPVRAYDVVVDGTEILV',
  1377032650408549891),
 (75,
  'hhhphhhphhhhphhphhhphpphhhh',
  'GAPTALPCVLPVRAYDVVVDGTEILVA',
  9687257153334895900),
 (76,
  'hhphhhphhhhphhphhhphpphhhhh',
  'APTALPCVLPVRAYDVVVDGTEILVAP',
  4002281547216460058)]

In [136]:
stitch_kmers(self_overlap)

'TGAPTALPCVLPVRAYDVVVDGTEILVAP'

In [137]:
other_overlap = match.get_overlapping_kmers(query)
other_overlap

[(2,
  'phhhphhhphhhhphhphhhphpphhh',
  'ELLVQVPGKAVVWEPLTAYVSFTNPLP',
  1377032650408549891),
 (3,
  'hhhphhhphhhhphhphhhphpphhhh',
  'LLVQVPGKAVVWEPLTAYVSFTNPLPV',
  9687257153334895900),
 (4,
  'hhphhhphhhhphhphhhphpphhhhh',
  'LVQVPGKAVVWEPLTAYVSFTNPLPVP',
  4002281547216460058)]

In [138]:
self_overlap_seq = stitch_kmers(self_overlap, use_encoded=False)
self_overlap_encoded = stitch_kmers(self_overlap, use_encoded=True)
other_overlap_seq = stitch_kmers(self_overlap, use_encoded=False)
other_overlap_encoded = stitch_kmers(other_overlap, use_encoded=True)

match_until_overlap = match.seq.index("TGAPTALPCVLPVRAYDVVVDGTEILVAP")

In [139]:
self_overlap_seq

'TGAPTALPCVLPVRAYDVVVDGTEILVAP'

In [140]:
query.seq

'TPELLVQVPGKAVVWEPLTAYVSFTNPLPVPLKGGVFTLEGAGLLSATQIHVNGAVAPSGKVSVKLSFSPMRTGVRKLLVDFDSDRLKDVKGVTTVVVHKK'

In [141]:
match.seq.index("TGAPTALPCVLPVRAYDVVVDGTEILVAP")

74

In [142]:
match.seq

'STPVRVATLDQLKPGVPTAFDVDGDEVMVVRDGDSVYAISNLCSHAEAYLDMGVFHAESLEIECPLHVGRFDVRTGAPTALPCVLPVRAYDVVVDGTEILVAPK'

## ITerate over some rows

In [105]:
query_scop_id_index = multisearch.columns.index("query_scop_id")
match_scop_id_index = multisearch.columns.index("match_scop_id")

for row in multisearch.head(10).iter_rows():
    query_id = row[query_scop_id_index]
    match_id = row[match_scop_id_index]

    query = SigSeq(sigs[query_id], sequences[query_id])
    match = SigSeq(sigs[match_id], sequences[match_id])

    query.display_alignment(match)


---
match_length 29
d1g0da3 b.1.5.1 (A:584-684) Transglutaminase, two C-terminal domains {Red sea bream (Chrysophrys major) [TaxId: 143350]}
                                                                        TPELLVQVPGKAVVWEPLTAYVSFTNPLPVPLKGGVFTLEGAGLLSATQIHVNGAVAPSGKVSVKLSFSPMRTGVRKLLVDFDSDRLKDVKGVTTVVVHKK
                                                                        phphhhphhhphhhhphhphhhphpphhhhhhphhhhphphhhhhphpphphphhhhhphphphphphphhpphhpphhhphpppphpphphhpphhhppp
                                                                          |||||||||||||||||||||||||||||
pphhphhphpphphhhhphhphphpphhhhpphpphhhhpphppphphhhphhhhphpphphpphhphhphphpphhhphhhphhhhphhphhhphpphhhhhp
STPVRVATLDQLKPGVPTAFDVDGDEVMVVRDGDSVYAISNLCSHAEAYLDMGVFHAESLEIECPLHVGRFDVRTGAPTALPCVLPVRAYDVVVDGTEILVAPK
d3gcea_ b.33.1.0 (A:) automated matches {Nocardioides aromaticivorans [TaxId: 200618]}

---
match_length 29
d5mdmc_ e.76.1.0 (C:) automated matches {Chandipura virus [TaxId: 11272]}
              